In [68]:
# Bibliotecas utilizadas
import requests
import os
import base64
from dotenv import load_dotenv
import pandas as pd
import json
from google.cloud import bigquery
from google.oauth2 import service_account

In [69]:
# Credenciais
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
id_data_hackers = os.getenv("ID_DATA_HACKERS")

In [70]:
# Token de autenticação
def get_Token():
    auth_string = str(base64.b64encode((client_id + ":" + client_secret).encode("utf-8")), "utf-8")

    url_login = 'https://accounts.spotify.com/api/token'
    headers = {
        "Authorization": "Basic " + auth_string,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    return requests.post(url_login, headers = headers, data = data).json()["access_token"]

In [71]:
# Obter token de autenticação
token = get_Token()
print(token)

BQCjNoS8z9dF71yWhuQ_zx4F9s1hXIJoz9QQNmSvgSKFykNn5nL43uuRHQRscvJs-3gP9PvGUP1fNJFwvYCbt__3zsfgVzHw0_mxlHdOMO9DA-tE-_uf


In [72]:
# Dados principais tabela 5
def getTabela5(termo, tipo, limite):
    auth_token = get_Token()
    params = {
        "q": termo,
        "type": tipo,
        "market": "BR",
        "limit": limite
    }

    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    return requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)


In [ ]:
# Total de episódios
def getTotalEpisodes(id):
    auth_token = get_Token()
    params = {
        "market": "BR"
    }
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }
    response = requests.get(f"https://api.spotify.com/v1/shows/{id}", params=params, headers=headers)
    rs = json.loads(json.dumps(response.json()))
    return rs["total_episodes"]

In [73]:
# teste
getTotalEpisodes('1oMIHOXsrLFENAeM743g93')

69

In [74]:
def tabela5():
    rs = getTabela5("Data Hackers", "episode", 50).json()
    episodes = getTotalEpisodes(id_data_hackers)
    items = rs["episodes"]["items"]
    data = [(item["id"], item["name"], item["description"], episodes) for item in items]
    df = pd.DataFrame(data, columns=["id", "name", "description", "total_episodes"])
    return df

In [ ]:
#tabela5()

In [77]:
# Busca pela lista de episódios
# limite entrega no máximo 20 linhas
# offset paginação

def getShowEpisodes(id, limite, offset):
    auth_token = get_Token()
    params = {
        "limit": limite,
        "offset": offset,
        "market": "BR"
    }
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }
    response = requests.get(f"https://api.spotify.com/v1/shows/{id}/episodes", params=params, headers=headers)
    return json.loads(json.dumps(response.json()))


In [78]:
def tabela6():
    offset = 0
    limite = 20
    episodios = getTotalEpisodes(id_data_hackers)
    data = []

    while offset < episodios:
        rs = getShowEpisodes(id_data_hackers, limite, offset)
        items = rs["items"]
        for item in items:
            data.append({
                "id": item["id"],
                "name": item["name"],
                "description": item["description"],
                "release_date": item["release_date"],
                "duration_ms": item["duration_ms"],
                "language": item["language"],
                "explicit": item["explicit"],
                "type": item["type"]
            })
        offset += limite
    
    df = pd.DataFrame(data)
    return df

In [82]:
#tabela6()

In [80]:
# Função principal
def main():
    # as 50 primeiras saidas pelo termo datahakers
    df5 = tabela5()
    # todos os epsodios
    df6 = tabela6()
    # todos os epsodios que tem boticário
    df7 = df6[df6['description'].str.contains('boticário', case=False)]
    key_path = "./GBQ.json"
    credentials = service_account.Credentials.from_service_account_file(key_path)
    df5.to_gbq(credentials=credentials, destination_table='BancoDW.Spotify_tbl_5', if_exists='replace')
    df6.to_gbq(credentials=credentials, destination_table='BancoDW.Spotify_tbl_6_tds_Ep', if_exists='replace')
    df7.to_gbq(credentials=credentials, destination_table='BancoDW.Spotify_tbl_7_Ep_gb', if_exists='replace')

In [81]:
# executa 
main()